In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
data_path = 'C:/Users/mccal/Desktop/Springboard Files/Springboard 26 SPARK SQL/Ex_Files_Spark_SQL_DataFrames/Exercise Files/Data'

json_df2_path = data_path+'/utilization.json'
df_util = spark.read.format('json').load(json_df2_path)

df_util.show(10)

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58|
|           0.53|03/05/2019 08:46:14|       0.35|      100|           62|
|           0.51|03/05/2019 08:51:14|        0.6|      100|           45|
+---------------+-------------------+-

In [2]:
df_util.createOrReplaceTempView("utilization")

In [3]:
df_util.count()

500000

In [6]:
df_util.describe().show()

+-------+-------------------+-------------------+-------------------+------------------+------------------+
|summary|    cpu_utilization|     event_datetime|        free_memory|         server_id|     session_count|
+-------+-------------------+-------------------+-------------------+------------------+------------------+
|  count|             500000|             500000|             500000|            500000|            500000|
|   mean| 0.6205177399999957|               null|0.37912809999999864|             124.5|          69.59616|
| stddev|0.15875173872912945|               null|0.15830931278376276|14.430884120553191|14.850676696352851|
|    min|               0.22|03/05/2019 08:06:14|                0.0|               100|                32|
|    max|                1.0|04/09/2019 01:22:46|               0.78|               149|               105|
+-------+-------------------+-------------------+-------------------+------------------+------------------+



In [7]:
df_util.stat.corr('cpu_utilization','free_memory')

-0.4704771573080754

In [8]:
df_util.stat.corr('session_count','free_memory')

-0.5008320848876573

In [10]:
df_util.stat.freqItems(('server_id', 'session_count')).show()

+--------------------+-----------------------+
| server_id_freqItems|session_count_freqItems|
+--------------------+-----------------------+
|[137, 146, 101, 1...|   [92, 101, 83, 104...|
+--------------------+-----------------------+



In [13]:
df_util_sample  = df_util.sample(fraction=0.05, withReplacement=False)
df_util_sample.count()

25106

In [16]:

spark.sql("SELECT min(cpu_utilization), max(cpu_utilization), stddev(cpu_utilization) FROM utilization").show()

+--------------------+--------------------+-----------------------+
|min(cpu_utilization)|max(cpu_utilization)|stddev(cpu_utilization)|
+--------------------+--------------------+-----------------------+
|                0.22|                 1.0|    0.15875173872912945|
+--------------------+--------------------+-----------------------+



In [18]:

spark.sql("SELECT server_id, min(cpu_utilization), max(cpu_utilization), stddev(cpu_utilization) \
           FROM utilization \
           GROUP BY server_id").show()

+---------+--------------------+--------------------+-----------------------+
|server_id|min(cpu_utilization)|max(cpu_utilization)|stddev(cpu_utilization)|
+---------+--------------------+--------------------+-----------------------+
|      112|                0.52|                0.92|    0.11528867845082576|
|      113|                0.58|                0.98|    0.11544345150353687|
|      130|                0.35|                0.75|    0.11568834774245991|
|      126|                0.48|                0.88|    0.11542612970702058|
|      149|                0.54|                0.94|    0.11543517500295467|
|      110|                0.35|                0.75|    0.11533251724450215|
|      136|                0.41|                 0.8|    0.11597405743182258|
|      144|                0.47|                0.87|    0.11478654960489501|
|      119|                0.22|                0.62|    0.11516031929842008|
|      116|                 0.3|                 0.7|    0.11506

In [19]:
spark.sql("SELECT server_id, FLOOR(cpu_utilization*100/10) bucket FROM utilization").show()

+---------+------+
|server_id|bucket|
+---------+------+
|      100|     5|
|      100|     4|
|      100|     5|
|      100|     5|
|      100|     3|
|      100|     4|
|      100|     5|
|      100|     4|
|      100|     5|
|      100|     5|
|      100|     3|
|      100|     6|
|      100|     6|
|      100|     5|
|      100|     2|
|      100|     4|
|      100|     4|
|      100|     6|
|      100|     4|
|      100|     5|
+---------+------+
only showing top 20 rows



In [20]:
spark.sql("SELECT count(*), FLOOR(cpu_utilization*100/10) bucket FROM utilization GROUP BY bucket ORDER BY bucket").show()

+--------+------+
|count(1)|bucket|
+--------+------+
|    8186|     2|
|   37029|     3|
|   68046|     4|
|  104910|     5|
|  116725|     6|
|   88242|     7|
|   56598|     8|
|   20207|     9|
|      57|    10|
+--------+------+

